# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
  ※本問題は仕様で **`visit_date` 昇順** が要求されるため最終的に付与
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）※本問では未使用
* 判定は **ID 連続**、表示は仕様どおりの列順

## 1) 問題

* `3 行以上の「連続した id」を持ち、かつ各行 people >= 100 のレコードを抽出する。結果は visit_date 昇順。`
* 入力: `Stadium(id int, visit_date date, people int)`（`visit_date` はユニーク、id 上昇とともに日付も上昇）
* 出力: `id, visit_date, people`（条件を満たす「連続 id の島」に属するすべての行）

## 2) 最適解（単一クエリ）

> まず `people >= 100` で縮小し、`id - ROW_NUMBER()` で **連番の島キー** を作成。
> その島の長さを `COUNT(*) OVER (PARTITION BY grp_key)` で求め、長さ ≥ 3 のみ残す。

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM stadium
  WHERE people >= 100
),
grp AS (
  SELECT
    id,
    visit_date,
    people,
    id - ROW_NUMBER() OVER (ORDER BY id) AS grp_key
  FROM pre
)
SELECT id, visit_date, people
FROM (
  SELECT
    g.*,
    COUNT(*) OVER (PARTITION BY grp_key) AS island_len
  FROM grp AS g
) x
WHERE island_len >= 3
ORDER BY visit_date;

Runtime 188 ms
Beats 58.33%

```

### 代替（`LAG` でブレーク検知 → 累積和で島採番）

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM stadium
  WHERE people >= 100
),
marked AS (
  SELECT
    id,
    visit_date,
    people,
    CASE WHEN id = LAG(id) OVER (ORDER BY id) + 1 THEN 0 ELSE 1 END AS is_break
  FROM pre
),
islands AS (
  SELECT
    id,
    visit_date,
    people,
    SUM(is_break) OVER (ORDER BY id) AS grp_key
  FROM marked
)
SELECT id, visit_date, people
FROM (
  SELECT
    i.*,
    COUNT(*) OVER (PARTITION BY grp_key) AS island_len
  FROM islands i
) s
WHERE island_len >= 3
ORDER BY visit_date;

Runtime 193 ms
Beats 47.52%
```

> Postgres はウィンドウ関数の **入れ子（window の中で別の window）** を許しませんが、上記はいずれも **段階的に計算**しているため問題ありません。

## 3) 要点解説

* **Gaps-and-Islands** の定石

  * `id - ROW_NUMBER()` が一定 ⇒ **連続 id の島**
  * その島のサイズを `COUNT() OVER (PARTITION BY grp_key)` で評価
* 先に `WHERE people >= 100` で絞ることで、ソート対象行を減らしウィンドウ計算を軽量化
* 並びは仕様に合わせ **`ORDER BY visit_date`**（`id` と日付の昇順は一致するが、仕様優先）

> インデックス（任意だが大規模なら推奨）
> `CREATE INDEX ON stadium (people, id);`
> 先頭 `people` で絞り、続く `id` 順でソート負荷を軽減

## 4) 計算量（概算）

* 対象行数を `n` とすると

  * ウィンドウ（`ROW_NUMBER` / `LAG`）: **O(n log n)**（`ORDER BY id` のソート）
  * `COUNT() OVER (PARTITION BY grp_key)`: **O(n)**（パーティション内の線形集計）
* 適切なインデックスにより実効コストはさらに低減

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Stadium] --> B[前処理 people >= 100]
  B --> C[ROW_NUMBER または LAG で島キー作成]
  C --> D[COUNT OVER で島サイズ算出]
  D --> E[島サイズ >= 3 を抽出]
  E --> F[visit_date 昇順で出力]
```

1. **JOIN/再集計を増やさずに “島の長さでフィルタ” している点は最適**
   　→ そのままでOK。CTEはPG16では基本インライン化されます（`MATERIALIZED`を付けない限り）。

2. **並び順の最適化**

* 仕様が許せば `ORDER BY id` に置換（`id ↑ ⇒ visit_date ↑` が保証されているため等価）。
  これで `visit_date` での**追加ソートを回避**でき、`people, id`系のインデックスから**ソート不要**で出せます。
* 仕様上どうしても `visit_date` 必須なら現行のまま。ただし対象行は「島長≥3」に絞られているのでコスト影響は限定的。

3. **インデックス設計（最重要）**
   実測時間の差は**インデックス有無・形**で大きく変わります。

```sql
-- people で範囲 → id の順序性を活かす
CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_stadium_people_id ON stadium (people, id);

-- “該当行だけ” を速く触りたいなら部分インデックス（最有力）
CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_stadium_hot ON stadium (id) WHERE people >= 100;

-- visit_date で最終ソートが厳しい場合のカバリング（任意・サイズと相談）
CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_stadium_people_id_date ON stadium (people, id, visit_date);
```

* **部分インデックス `(id) WHERE people >= 100`** は今回のフィルタにドンピシャ。
  `pre` の抽出・`ORDER BY id` のウィンドウ両方が軽くなります。
* すでに巨大データなら `ANALYZE stadium;` で統計を最新化。

---

## 速度最優先の等価クエリ（`ORDER BY id` 可の場合）

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM stadium
  WHERE people >= 100
),
grp AS (
  SELECT
    id,
    visit_date,
    people,
    id - ROW_NUMBER() OVER (ORDER BY id) AS grp_key
  FROM pre
)
SELECT id, visit_date, people
FROM (
  SELECT
    g.*,
    COUNT(*) OVER (PARTITION BY grp_key) AS island_len
  FROM grp AS g
) x
WHERE island_len >= 3
ORDER BY id;  -- 許されるならこちらが速い

Runtime 184 ms
Beats 69.15%

```

* `ix_stadium_hot`（部分インデックス）があると、**範囲抽出＋連番ウィンドウ**が非常に効きます。
* ほぼ同じ書きぶりで、最終ソート回避による**数〜数十%の短縮**が見込めます（データ分布次第）。

---

## ウィンドウ不使用の高速代替（自己相関 `EXISTS` 三点チェック）

> 「島に属する行」を、“自分の前後に連続が2つあるか” の**3パターン**で判定。
> ウィンドウが重い分布ではこちらが速いことがあります（インデックス前提）。

```sql
SELECT s.id, s.visit_date, s.people
FROM stadium s
WHERE s.people >= 100
AND (
  -- s, s-1, s-2 が連続
  EXISTS (
    SELECT 1 FROM stadium a
    WHERE a.id = s.id - 1 AND a.people >= 100
      AND EXISTS (
        SELECT 1 FROM stadium b
        WHERE b.id = s.id - 2 AND b.people >= 100
      )
  )
  -- s, s+1, s+2 が連続
  OR EXISTS (
    SELECT 1 FROM stadium a
    WHERE a.id = s.id + 1 AND a.people >= 100
      AND EXISTS (
        SELECT 1 FROM stadium b
        WHERE b.id = s.id + 2 AND b.people >= 100
      )
  )
  -- s-1, s, s+1 が連続（中央）
  OR (
    EXISTS (SELECT 1 FROM stadium a WHERE a.id = s.id - 1 AND a.people >= 100)
    AND EXISTS (SELECT 1 FROM stadium b WHERE b.id = s.id + 1 AND b.people >= 100)
  )
)
ORDER BY /* visit_date */ id;  -- 仕様に合わせて変更

Runtime 476 ms
Beats 5.26%

```

**ポイント**

* 連続IDの島（長さ≥3）に属する行は、上記いずれかの三条件を満たします。
* インデックスは **`(id)` の部分インデックス `WHERE people >= 100`** が効きます。
* ウィンドウなし・ソート最小化により、**高選択度**かつ**疎な分布**では特に有利。

---

## 追加の微調整

* `pre`/`grp` はPG16では**自動インライン**が既定。重い場合のみ

  * 再利用が多い中間を強制マテリアライズ → `WITH ... AS MATERIALIZED`
  * 逆にインラインしたい → `WITH ... AS NOT MATERIALIZED`（既定なので通常不要）
* 実行直前にだけ `SET work_mem = '64MB';` などで**ソート用メモリ**を増やすと、ディスク落ち回避で安定短縮。

---

### まとめ

* まずは **部分インデックス `(id) WHERE people >= 100`** を作成
* 仕様が許すなら **`ORDER BY id`** に変更（無理なら現行維持）
* データ分布次第で **EXISTS 版** も計測し、速い方を採用

この3点で、提示の **188ms → 1〜3割程度の短縮**が現実的に狙えます（環境・分布依存）。

